In [ ]:
pip install PyPDF2 transformers torch

In [ ]:
import re
import PyPDF2
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

def extract_and_summarize(file_path):
    if not file_path.endswith('.pdf'):
        return 'Please convert file to PDF.'

    # Open the PDF file
    with open(file_path, 'rb') as file:
        # Create a PDF reader object
        reader = PyPDF2.PdfReader(file)

        # Extract text from each page
        extracted_text = []
        for page in reader.pages:
            extracted_text.append(page.extract_text())

    # Define start and end phrases
    start_phrases = ["recommendation", "lessons learned", "advice to planning authorities"]
    end_phrases = ["reference", "appendix", "annex", "list of", "conclusion", "bibliography", "works cited",
                   "introduction", "board member statements", "executive summary", "abbreviations and acronyms"]

    # Extract the relevant part of the document
    extracted_part = []
    found_start = False
    for line in extracted_text:
        if any(phrase in line.lower() for phrase in start_phrases):
            found_start = True
        if found_start:
            extracted_part.append(line)
            if any(phrase in line.lower() for phrase in end_phrases):
                break

    # Extract sentences containing specific keywords
    keywords = ["he", "she", "they", "I", "user", "operator", "manager", "management", "team", "lead", "leader",
                "inspector", "mechanic", "engineer", "driver", "pilot", "crew", "worker", "contractor", "operative"]
    extracted_sentences = []
    for line in extracted_text:
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', line)
        for sentence in sentences:
            if any(keyword in sentence.lower() for keyword in keywords):
                extracted_sentences.append(sentence)

    combined_text = ' '.join(extracted_part + extracted_sentences)

    # Load BART model and tokenizer
    model_name = 'facebook/bart-large-cnn'
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)

    def summarize_text(text, max_length=150):
        inputs = tokenizer.encode("summarize: " + text, return_tensors='pt', max_length=1024, truncation=True)
        summary_ids = model.generate(inputs, max_length=max_length, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

    # Split the text into 1000 token chunks
    tokens = tokenizer(combined_text, return_tensors='pt')['input_ids'][0]
    chunk_size = 1000
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]

    # Summarize each chunk
    summaries = [summarize_text(tokenizer.decode(chunk, skip_special_tokens=True)) for chunk in chunks]

    # Iteratively summarize until the final combined summaries text is under 500 tokens
    combined_summary = ' '.join(summaries)
    while len(tokenizer(combined_summary)['input_ids']) > 500:
        combined_summary = summarize_text(combined_summary)

    return combined_summary

file_path = 'your_file.pdf'
result = extract_and_summarize(file_path)
print(result)